Package installation

In [1]:
!pip install #THINGS# to install packages that are needed to be access later 

ERROR: You must give at least one requirement to install (see "pip help install")


In [2]:
!pip install tflearn
!pip install nltk
!pip install punkt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 107 kB 20.6 MB/s 
  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127299 sha256=0865981004b28848752c1e73f7c4d0272f113bdf2bea75af7d6c4efec3f5d967
  Stored in directory: /root/.cache/pip/wheels/5f/14/2e/1d8e28cc47a5a931a2fb82438c9e37ef9246cc6a3774520271
Successfully built tflearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement punkt (from versions: none)
ERROR: No matching distribution found for punkt


Imports

In [3]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
from tensorflow.python.framework import ops
import numpy
import tflearn
import tensorflow
import random


#loads json data
#json file is our responses to certain tags
import json
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Instructions for updating:
non-resource variables are not supported in the long term


In [7]:
from google.colab import files
uploaded = files.upload()
tempData = next(iter(uploaded.values()))

Saving intents.json to intents (1).json


Load in data

In [8]:
data = json.loads(tempData.decode())

In [9]:
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
  words = []
  labels = []
  docs_x = []
  docs_y = []
  #extracting data
  for intent in data["intents"]:
    for pattern in intent["patterns"]:
      wrds = nltk.word_tokenize(pattern)
      words.extend(wrds)
      docs_x.append(wrds)
      docs_y.append(intent["tag"])
    
    if intent["tag"] not in labels:
      labels.append(intent["tag"])

  words = [stemmer.stem(w.lower()) for w in words if w != "?"]
  words = sorted(list(set(words)))

  labels = sorted(labels)

  training = []
  output = []

  out_empty = [0 for _ in range(len(labels))]

  for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
      if w in wrds:
        bag.append(1)
      else:
        bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

  training = numpy.array(training)
  output = numpy.array(output)


Train Model

In [10]:
#developing model (neural network)
#tensorflow.reset_default_graph()
ops.reset_default_graph
net = tflearn.input_data(shape=[None,len(training[0])])
net = tflearn.fully_connected(net, 2)
net = tflearn.fully_connected(net, 2)
net = tflearn.fully_connected(net, len(output[0]), activation = "softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [11]:
model.fit(training, output, n_epoch=70, batch_size=3, show_metric= True)
model.save("model.tflearn")

Training Step: 839  | total loss: 0.96608 | time: 0.047s
| Adam | epoch: 070 | loss: 0.96608 - acc: 0.6735 -- iter: 33/34
Training Step: 840  | total loss: 0.99658 | time: 0.050s
| Adam | epoch: 070 | loss: 0.99658 - acc: 0.7062 -- iter: 34/34
--


Ignore this section (debugging)

In [ ]:
#training and save model
try:
  model.load("model.tflearn")
except:
  model.fit(training, output, n_epoch=2, batch_size=2, show_metric = True)
  model.save("model.tflearn")

Define methods

In [ ]:
#making predictions
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)

In [ ]:
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

New tokenized version? (in progress)

In [ ]:
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        if tag == 'comparison':
            if 'O(n)' in inp:
                if 'O(n^2)' in inp:
                    responses = 'O(n) will be faster than O(n^2)'
                elif 'O(log(n))' in inp:
                    responses = 'O(n) will be slower than O(log(n))'

        # etc etc. till all cases fulfilled?            

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

Run Chatbot

In [ ]:
chat()

Start talking with the bot (type quit to stop)!
You: hi
Good to see you again
You: bye
Hello, thanks for visiting
You: thanks
Good to see you again
You: thanks
Hello, thanks for visiting
You: thank you
Good to see you again
You: quit
